<h3 style="color:blue;">Tratamento e Construção de Dashboard Completo</h3>
<h4 style="color:graywhite;">No campo abaixo preencha com as informações e arquivos coletados, em seguida pressione "Run All"</h4>

In [163]:
# Informações e Arquivos coletados
Rotação = 2500
Tolerância = 10
regime = 'rotação crescente em carga máxima' 

Balança_Micheletti_Inferior  =  'balançainferior_ensaio_30c_2500rpm_16082024.xls'
Balança_Precisão_Superior =  'balançasuperior_ensaio_30c_2500rpm_16082024.xls'
Dinamometro = '2500 RPM.csv'
Termopar = 'TESTE 2500 RPM_16082024__30C.csv'


In [164]:
#  Preparação - Balança Micheletti (inferior)

import pandas as pd
import re
import os

# Nome do arquivo xls
file_bmicheletti = Balança_Micheletti_Inferior

# Pasta onde o arquivo Excel está localizado
pasta_origem = r'C:\Área de Trabalho\Automação IPBEN\EA211\Dados coletados - Balança'

# Caminho completo para o arquivo xls
caminho_arquivo = os.path.join(pasta_origem, file_bmicheletti)

# Nome do arquivo Excel
nome_arquivo_excel = 'p_balança_micheletti.xlsx'

# Pasta para salvar os arquivos Excel
pasta_destino = r'C:\Área de Trabalho\Automação IPBEN\EA211\Arquivos Intermediários'

# Caminho completo para salvar o arquivo Excel
caminho_arquivo_excel = os.path.join(pasta_destino, nome_arquivo_excel)

# Ler o arquivo e processar manualmente
data = []
with open(caminho_arquivo, 'r') as file:
    for line in file:
        # Dividir a linha usando letras maiúsculas como delimitadores
        values = [v.strip() for v in line.split() if v.strip()]
        data.append(values)

# Criar um DataFrame com os dados processados
df = pd.DataFrame(data)

# Remover a primeira coluna (coluna 1)
if df.columns[0] == 'Unnamed: 0':
    df.drop(columns=[0], inplace=True)

# Definir os nomes das colunas
col_names = ['Tempo', 'Peso Bruto', 'Peso (B Micheletti)', 'Tara']

# Verificar o número de colunas restantes e renomear as colunas
num_cols = len(df.columns)
if num_cols >= 3:
    col_names = ['Tempo'] + col_names[1:num_cols]
    df.columns = col_names
else:
    print("Erro: O DataFrame não tem colunas suficientes para renomear todas as colunas necessárias.")

# Remover a linha 1 (índice 0)
df.drop(index=[0], inplace=True)

# Limpar e converter os dados nas colunas específicas
df['Peso Bruto'] = df['Peso Bruto'].apply(lambda x: re.search(r'\d+,\d+', str(x)).group(0) if re.search(r'\d+,\d+', str(x)) else x)
df['Peso (B Micheletti)'] = df['Peso (B Micheletti)'].apply(lambda x: re.search(r'\d+,\d+', str(x)).group(0) if re.search(r'\d+,\d+', str(x)) else x)
df['Tara'] = df['Tara'].apply(lambda x: re.search(r'\d+,\d+', str(x)).group(0) if re.search(r'\d+,\d+', str(x)) else x)
df['Tempo'] = df['Tempo'].apply(lambda x: str(x).split(',')[0] if isinstance(x, str) else x)

# Inserir uma linha abaixo do título com as informações desejadas
nova_linha = pd.DataFrame([['seg', 'kg', 'kg', 'kg']], columns=col_names)
df = pd.concat([df.iloc[:0], nova_linha, df.iloc[0:]]).reset_index(drop=True)

# Converter as colunas 'Peso Bruto', 'Peso Líquido' e 'Tara' para valores numéricos a partir da linha 2 (índice 1)
for col in ['Tempo', 'Peso Bruto', 'Peso (B Micheletti)', 'Tara']:
    df.loc[1:, col] = pd.to_numeric(df.loc[1:, col].str.replace(',', '.'), errors='coerce')

# Remover a linha 2 (índice 1)
df.drop(index=[1], inplace=True)
df.drop(index=[2], inplace=True)

# Mostrar o DataFrame carregado e atualizado
print("\nDataFrame carregado e atualizado:")
print(df)

# Salvar o DataFrame como um arquivo .xlsx com os nomes das colunas na primeira linha
df.to_excel(caminho_arquivo_excel, index=False, header=True)

print(f"\nDados salvos em {nome_arquivo_excel}")


DataFrame carregado e atualizado:
    Tempo Peso Bruto Peso (B Micheletti) Tara
0     seg         kg                  kg   kg
3       2      8.605               8.605  0.0
4       3      8.605               8.605  0.0
5       4        8.6                 8.6  0.0
6       5        8.6                 8.6  0.0
..    ...        ...                 ...  ...
132   131       8.52                8.52  0.0
133   132       8.52                8.52  0.0
134   133       8.52                8.52  0.0
135   134       8.52                8.52  0.0
136   135       8.52                8.52  0.0

[135 rows x 4 columns]

Dados salvos em p_balança_micheletti.xlsx


In [165]:
# Preparação - Balança de Precisão (superior)

import pandas as pd
import os

# Nome do arquivo xls
nome_arquivo_xls = Balança_Precisão_Superior

# Pasta onde o arquivo XLS está localizado
pasta_origem = r'C:\Área de Trabalho\Automação IPBEN\EA211\Dados coletados - Balança'

# Nome e pasta para salvar o DataFrame tratado em um novo arquivo Excel
nome_arquivo_excel = 'p_balança_precisao.xlsx'
pasta_destino = r'C:\Área de Trabalho\Automação IPBEN\EA211\Arquivos Intermediários'
caminho_arquivo_excel = os.path.join(pasta_destino, nome_arquivo_excel)

# Caminho completo para o arquivo XLS
caminho_arquivo = os.path.join(pasta_origem, nome_arquivo_xls)

# Ler o arquivo e processar manualmente
data = []
with open(caminho_arquivo, 'r') as file:
    for line in file:
        # Dividir a linha usando letras maiúsculas como delimitadores
        values = [v.strip() for v in line.split() if v.strip()]
        data.append(values)

# Criar um DataFrame com os dados processados
df = pd.DataFrame(data)

# Apagar colunas específicas: 2, 3, 4, 6, 7 e 9 (baseado em zero)
colunas_para_remover = [1, 2, 3, 5, 6, 8]  # Índices ajustados para zero
df.drop(columns=df.columns[colunas_para_remover], inplace=True)


# Definir os nomes das colunas
df.columns = ['Tempo', 'Tara', 'Peso (B Precisão)']

# Apagar linhas com índices 0 e 1
df = df.drop(index=[0, 1])

# Inserir uma linha após o título com os valores 'seg', 'kg', 'kg'
nova_linha = pd.DataFrame([['seg', 'kg', 'kg']], columns=df.columns)
df = pd.concat([nova_linha, df], ignore_index=True)

# Apagar todo o conteúdo após a vírgula na coluna 'Tempo'
df['Tempo'] = df['Tempo'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

# Converter todos os valores a partir da linha com o índice 2 para valores numéricos
for col in df.columns:
    df.loc[1:, col] = pd.to_numeric(df.loc[1:, col], errors='coerce')

# Dividir todos os valores da coluna 'Tara' por 1000 a partir da linha 2
df.loc[1:, 'Peso (B Precisão)'] = df.loc[1:, 'Peso (B Precisão)'] / 1000

# Mostrar o DataFrame carregado e atualizado
print("\nDataFrame carregado e atualizado:")
print(df)

# Salvar o DataFrame como um arquivo .xlsx com os nomes das colunas na primeira linha
df.to_excel(caminho_arquivo_excel, index=False, header=True)

print(f"\nDados salvos em {nome_arquivo_excel}")


DataFrame carregado e atualizado:
    Tempo Tara Peso (B Precisão)
0     seg   kg                kg
1       1  0.0            4.9639
2       2  0.0           4.96211
3       3  0.0           4.95976
4       4  0.0           4.95714
..    ...  ...               ...
131   131  0.0           4.67257
132   132  0.0           4.67057
133   133  0.0           4.66806
134   134  0.0           4.66612
135   135  0.0           4.66348

[136 rows x 3 columns]

Dados salvos em p_balança_precisao.xlsx


In [166]:
# Preparação - Junção Balanças

import pandas as pd
import os

# Caminhos para os arquivos Excel
arquivo_excel_1 = r'C:\\Área de Trabalho\\Automação IPBEN\\EA211\\Arquivos Intermediários\\p_balança_precisao.xlsx'
arquivo_excel_2 = r'C:\\Área de Trabalho\\Automação IPBEN\\EA211\\Arquivos Intermediários\\p_balança_micheletti.xlsx'

# Caminho para o novo arquivo Excel
arquivo_excel_novo = r'C:\\Área de Trabalho\\Automação IPBEN\\EA211\\Arquivos Intermediários\\p_balança.xlsx'

# Ler os arquivos Excel
df1 = pd.read_excel(arquivo_excel_1)
df2 = pd.read_excel(arquivo_excel_2)

# Unir os DataFrames com base na coluna "Tempo"
df_combinado = pd.merge(df1, df2, on='Tempo', how='outer')

# Apagar colunas específicas: 2, 4 e 6 (baseado em zero)
colunas_para_remover = [1, 3, 5]  # Índices ajustados para zero
df_combinado.drop(columns=df_combinado.columns[colunas_para_remover], inplace=True)
df_combinado = df_combinado.drop(df_combinado.index[-1])


# Salvar o DataFrame combinado em um novo arquivo Excel
df_combinado.to_excel(arquivo_excel_novo, index=False)

print(f'Dados combinados salvos em p_balanças.xlsx')

Dados combinados salvos em p_balanças.xlsx


In [167]:
# Preparação - Dinamômetro
import pandas as pd
import numpy as np
import os

# Nome do arquivo CSV
nome_arquivo_csv = Dinamometro

# Pasta onde o arquivo CSV está localizado
pasta_origem = 'C:\Área de Trabalho\Automação IPBEN\EA211\Dados coletados - Dinamometro'

# Caminho completo para o arquivo CSV
caminho_arquivo = os.path.join(pasta_origem, nome_arquivo_csv)

# Função para identificar os conjuntos de dados no arquivo CSV, iniciando da linha 16
def separar_dados(lines):
    data1 = []
    data2 = []
    current_data = data1
    start_second_dataset = False
    
    for line in lines[14:]:  # Começa a leitura a partir da linha 16
        stripped_line = line.strip()
        if not stripped_line:  # Linhas em branco mudam o conjunto de dados
            current_data = data2
            start_second_dataset = True
        else:
            if start_second_dataset:
                # Pular a primeira linha da segunda base de dados
                start_second_dataset = False
                continue
            current_data.append(stripped_line)
    
    # Se não houver linha em branco, data2 será vazio; nesse caso, devemos copiar data1 para data2
    if not data2:
        data2 = data1[:]
    
    return data1, data2

# Caminho para o arquivo CSV (altere para o caminho correto do seu arquivo)
file_path = caminho_arquivo

# Ler o arquivo CSV
with open(file_path, 'r', encoding='latin1') as file:
    lines = file.readlines()

# Separar os conjuntos de dados, começando da linha 16
data1, data2 = separar_dados(lines)

# Verificar se temos dados suficientes em ambos os conjuntos
if not data1 or not data2:
    raise ValueError("Um ou ambos os conjuntos de dados estão vazios. Verifique a formatação do arquivo CSV.")

# Extrair e validar cabeçalhos
header1 = data1.pop(0).split(';')
unidades_df1 = data1.pop(0).split(';')
header2 = data2.pop(0).split(';')

# Função para ajustar o número de colunas
def ajustar_colunas(data, header):
    max_columns = len(header)
    adjusted_data = []
    for row in data:
        columns = row.split(';')
        # Preencher com NaN se houver menos colunas do que o esperado
        if len(columns) < max_columns:
            columns.extend([np.nan] * (max_columns - len(columns)))
        # Ignorar ou tratar linhas com mais colunas do que o esperado
        elif len(columns) > max_columns:
            columns = columns[:max_columns]
        adjusted_data.append(columns)
    return adjusted_data

# Ajustar os dados para garantir que o número de colunas corresponda ao cabeçalho
data1 = ajustar_colunas(data1, header1)
data2 = ajustar_colunas(data2, header2)

# Converter listas de linhas em DataFrames
df1 = pd.DataFrame(data1, columns=header1)
df2 = pd.DataFrame(data2, columns=header2)

# Renomear a primeira coluna para 'TEMPO'
df1 = df1.rename(columns={header1[0]: 'Tempo'})
df2 = df2.rename(columns={header2[0]: 'Tempo'})

# Remover espaços em branco dos nomes das colunas
df1.columns = df1.columns.str.strip()
df2.columns = df2.columns.str.strip()

# Verificar se a coluna Tempo está presente em ambos os DataFrames
if 'Tempo' not in df1.columns or 'Tempo' not in df2.columns:
    raise KeyError("A coluna 'Tempo' não está presente em um dos DataFrames.")

# Converter a coluna de tempo para o tipo numérico
df1['Tempo'] = pd.to_numeric(df1['Tempo'], errors='coerce')
df2['Tempo'] = pd.to_numeric(df2['Tempo'], errors='coerce')

# Converter outras colunas numéricas para o tipo numérico
for col in df1.columns:
    if col != 'Tempo':
        df1[col] = pd.to_numeric(df1[col], errors='coerce')

for col in df2.columns:
    if col != 'Tempo':
        df2[col] = pd.to_numeric(df2[col], errors='coerce')

# Mesclar os conjuntos de dados usando a coluna de tempo como referência
merged_df = pd.merge(df2, df1, on='Tempo', how='inner')

# Extraindo unidades do segundo DataFrame
unidades_df2 = [col.split('(')[-1].replace(')', '') if '(' in col else '' for col in header2]
# Adicionando 'Tempo' para a unidade de tempo
unidades_df2[0] = 'seg'

# Inserir a linha de unidades de medida no DataFrame mesclado
unidades_merged = [unidades_df2[0]] + unidades_df2[1:] + unidades_df1[1:]
merged_df.loc[-1] = unidades_merged  # Insere a linha de unidades
merged_df.index = merged_df.index + 1  # Ajusta o índice
merged_df = merged_df.sort_index()  # Reordena o DataFrame

# Identificar e remover colunas com títulos terminados em '_x' ou '_y'
cols_to_drop = [col for col in merged_df.columns if col.endswith('_x') or col.endswith('_y')]
merged_df = merged_df.drop(columns=cols_to_drop)
merged_df = merged_df.drop(merged_df.index[1:160])

# Salvar o DataFrame mesclado em um novo arquivo Excel
merged_df.to_excel('C:\Área de Trabalho\Automação IPBEN\EA211\Arquivos Intermediários/p_dinamometro.xlsx', index=False)

print("Arquivo 'p_dinamometro.xlsx' salvo com sucesso.")

<>:10: SyntaxWarning:

invalid escape sequence '\A'

<>:127: SyntaxWarning:

invalid escape sequence '\A'

<>:10: SyntaxWarning:

invalid escape sequence '\A'

<>:127: SyntaxWarning:

invalid escape sequence '\A'

C:\Users\Lucas\AppData\Local\Temp\ipykernel_19468\2102802660.py:10: SyntaxWarning:

invalid escape sequence '\A'

C:\Users\Lucas\AppData\Local\Temp\ipykernel_19468\2102802660.py:127: SyntaxWarning:

invalid escape sequence '\A'



Arquivo 'p_dinamometro.xlsx' salvo com sucesso.


In [168]:
# Preparação - Temperaturas e Pressões

import pandas as pd
import os

# Nome do arquivo CSV
nome_arquivo_csv = Termopar

# Pasta onde o arquivo CSV está localizado
pasta_origem = 'C:\\Área de Trabalho\\Automação IPBEN\\EA211\\Dados coletados - Termopar'

# Nome do arquivo Excel
nome_arquivo_excel = 'p_termopar.xlsx'

# Pasta para salvar os arquivos Excel
pasta_destino = 'C:\\Área de Trabalho\\Automação IPBEN\\EA211\\Arquivos Intermediários'

# Caminho completo para o arquivo CSV
caminho_arquivo = os.path.join(pasta_origem, nome_arquivo_csv)

# Lendo o arquivo CSV
dados = pd.read_csv(caminho_arquivo)

# Adicionando a coluna "Ordem" com a contagem de linhas
dados.insert(0, 'Tempo', range(1, len(dados) + 1))

# Adicionando as unidades nas colunas específicas
unidades = ['seg'] + ['ºC', 'ºC', 'ºC', 'bar', 'bar'] + [''] * (dados.shape[1] - 6)
dados.loc[-1] = unidades  # Adiciona a linha no topo do DataFrame
dados.index = dados.index + 1  # Ajusta os índices para empurrar os dados originais para baixo
dados = dados.sort_index()  # Ordena os índices

# Exibindo as primeiras linhas para verificar se os dados foram carregados corretamente
print(dados.head())

# Caminho completo para salvar o arquivo Excel
caminho_arquivo_excel = os.path.join(pasta_destino, nome_arquivo_excel)

# Salvando os dados em um arquivo Excel na pasta de destino
dados.to_excel(caminho_arquivo_excel, index=False)

print(f"Arquivo Excel salvo em: {caminho_arquivo_excel}")

  Tempo Temperatura Entrada Turbina Temperatura Saida Turbina  \
0   seg                          ºC                        ºC   
1     1                      632.48                    468.15   
2     2                      634.99                    470.78   
3     3                      636.75                    472.62   
4     4                      637.75                     475.5   

  Temperatura Saida Catalisador Pressao coletor de escape  \
0                            ºC                       bar   
1                        444.58                      4.09   
2                        446.84                      4.09   
3                         449.1                      3.48   
4                        451.35                      3.48   

  Pressao Saida Turbina  
0                   bar  
1                 -0.18  
2                  1.04  
3                 -0.18  
4                  1.04  
Arquivo Excel salvo em: C:\Área de Trabalho\Automação IPBEN\EA211\Arquivos Intermediár

In [169]:
# Dashboard

import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, State
import os
import plotly.graph_objects as go

# Configurações iniciais
inicio = Rotação
tolerancia = Tolerância
final = inicio # São iguais em caso de rotação única
intervalos_rotacao = 500

# Caminhos dos arquivos
arquivo_dinamometro = 'p_dinamometro.xlsx'
arquivo_termopar = 'p_termopar.xlsx'
arquivo_balanca = 'p_balança.xlsx'
pasta_origem = 'C:\\Área de Trabalho\\Automação IPBEN\\EA211\\Arquivos Intermediários'
nome_arquivo_excel = arquivo_dinamometro
caminho_arquivo = os.path.join(pasta_origem, nome_arquivo_excel)

# Leitura e limpeza dos dados
df = pd.read_excel(caminho_arquivo)
df.columns = df.columns.str.strip()
df['ROTACAO (RPM)'] = pd.to_numeric(df['ROTACAO (RPM)'], errors='coerce')

# Função para criar categorias de rotação
def criar_categorias(inicio, final, intervalos_rotacao, tolerancia):
    pontos = range(inicio, final + intervalos_rotacao, intervalos_rotacao)
    categorias = [{'label': f'{ponto}', 'value': f'{ponto-tolerancia}-{ponto+tolerancia}'} for ponto in pontos]
    categorias.append({'label': 'Tudo', 'value': 'outros'})
    return categorias

# Inicializar a aplicação Dash
app = Dash(__name__)

# Estilo customizado para a dropdown multipla
dropdown_style = {
    'width': '50%',
    'padding': '10px',
    'borderRadius': '5px',
    'backgroundColor': '#f9f9f9',
    'color': '#333',
    'outline': 'none',
    'border': '1px solid #ccc',
    'cursor': 'pointer'
}

# Leitura e limpeza dos dados do Termopar
novo_nome_arquivo_excel_1 = arquivo_termopar
novo_caminho_arquivo_1 = os.path.join(pasta_origem, novo_nome_arquivo_excel_1)
df_novo_1 = pd.read_excel(novo_caminho_arquivo_1)
df_novo_1.columns = df_novo_1.columns.str.strip()

# Leitura e limpeza dos dados da Balança
novo_nome_arquivo_excel_2 = arquivo_balanca
novo_caminho_arquivo_2 = os.path.join(pasta_origem, novo_nome_arquivo_excel_2)
df_novo_2 = pd.read_excel(novo_caminho_arquivo_2)
df_novo_2.columns = df_novo_2.columns.str.strip()

# Calcular a diferença entre o primeiro e o último valor da coluna 2 da balança
selected_data1 = pd.to_numeric(df_novo_2['Peso (B Precisão)'], errors='coerce')  # Convertendo para numérico
first_value1 = selected_data1.iloc[2]
last_value1 = selected_data1.iloc[-1]
difference1 = first_value1 - last_value1
difference_l1 = round(difference1, 3)
consumo1 = difference1 * 1.1351
consumo_l1 = round(consumo1, 3)

# Calcular a diferença entre o primeiro e o último valor da coluna 3 da balança
#selected_data2 = pd.to_numeric(df_novo_2['Peso (B Micheletti)'], errors='coerce')  # Convertendo para numérico
#first_value2 = selected_data2.iloc[2]
#last_value2 = selected_data2.iloc[-1]
#difference2 = first_value2 - last_value2
#difference_l2 = round(difference2, 3)
#consumo2 = difference2 * 1.1351
#consumo_l2 = round(consumo2, 3)

# Layout da aplicação
app.layout = html.Div(style={'backgroundColor': '#f0f0f0'}, children=[
    html.H1(children=(f'EA211 em  {regime} a {inicio} rpm')),
    html.Div([
        dcc.Graph(id='scatter-plot-1'),
        dcc.Dropdown(
            id='column-dropdown-1',
            options=[{'label': col, 'value': col} for col in df.columns],
            value=df.columns[0],
            clearable=False,
            style={'width': '50%'}
        ),
        html.Div(id='statistics-1')  # Container para exibir estatísticas
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Div([
        dcc.Graph(id='scatter-plot-2'),
        dcc.Dropdown(
            id='column-dropdown-2',
            options=[{'label': col, 'value': col} for col in df.columns],
            value=df.columns[0],
            clearable=False,
            style={'width': '50%'}
        ),
        html.Div(id='statistics-2')  # Container para exibir estatísticas
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Br(),
    html.Label('Intervalo de Rotação:'),
    dcc.Dropdown(
        id='rotation-interval-dropdown',
        options=criar_categorias(inicio, final, intervalos_rotacao, tolerancia),
        value=[f'{inicio-tolerancia}-{inicio+tolerancia}'],
        multi=True,
        style=dropdown_style
    ),

    html.Div([
        dcc.Graph(id='scatter-plot-3'),
        dcc.Dropdown(
            id='column-dropdown-3',
            options=[{'label': col, 'value': col} for col in df_novo_1.columns],
            value=df_novo_1.columns[0],
            clearable=False,
            style={'width': '50%'}
        ),
        html.Div(id='statistics-3')  # Container para exibir estatísticas
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'marginTop': '20px'}),
    
    html.Div([
        dcc.Graph(id='scatter-plot-4'),
        dcc.Dropdown(
            id='column-dropdown-4',
            options=[{'label': col, 'value': col} for col in df_novo_2.columns],
            value=df_novo_2.columns[0],
            clearable=False,
            style={'width': '50%'}
        ),
        html.Div([
            html.Label(f'Consumo de combustível (Precisão): {difference_l1}kg ou {consumo_l1}l', style={'fontSize': '18px', 'fontWeight': 'bold','color': '#1f77b4'}),  # Exibir a diferença calculada
            html.Br(),
            #html.Label(f'Consumo de combustível (Micheletti): {difference_l2}kg ou {consumo_l2}l', style={'fontSize': '18px', 'fontWeight': 'bold','color': '#1f77b4'}),  # Exibir a diferença calculada
            #html.Br(),
            html.Label(f'Considerando a massa específica do etanol em 809kg/m³')
        ])
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'marginTop': '20px'})
])

# Callback para atualizar os gráficos
@app.callback(
    [Output('scatter-plot-1', 'figure'),
     Output('scatter-plot-2', 'figure'),
     Output('scatter-plot-3', 'figure'),
     Output('scatter-plot-4', 'figure')],
    [Input('column-dropdown-1', 'value'),
     Input('column-dropdown-2', 'value'),
     Input('column-dropdown-3', 'value'),
     Input('column-dropdown-4', 'value'),
     Input('rotation-interval-dropdown', 'value')]
)
def update_graph(selected_column_1, selected_column_2, selected_column_3, selected_column_4, selected_categories):
    if not selected_categories:
        return {}, {}, {}, {}

    conditions = [df['ROTACAO (RPM)'].between(*map(int, category.split('-'))) for category in selected_categories if category != 'outros']
    if 'outros' in selected_categories:
        if conditions:
            selected_data = df[~np.logical_or.reduce(conditions)]
        else:
            selected_data = df.copy()
    else:
        selected_data = df[np.logical_or.reduce(conditions)]

    # Gráfico 1
    fig_1 = px.scatter(selected_data, x='Tempo', y=selected_column_1, title=f'{selected_column_1} por Tempo', labels={'Tempo': 'segundos', selected_column_1: selected_column_1})
    fig_1 = px.scatter(selected_data, x='Tempo', y=selected_column_1, title=f'{selected_column_1} por Tempo', labels={'Tempo': 'segundos', selected_column_1: selected_column_1}, trendline='ols')
    fig_1.update_traces(marker=dict(size=3), marker_color='darkblue')

    # Gráfico 2
    fig_2 = px.scatter(selected_data, x='Tempo', y=selected_column_2, title=f'{selected_column_2} por Tempo', labels={'Tempo': 'segundos', selected_column_2: selected_column_2})
    fig_2 = px.scatter(selected_data, x='Tempo', y=selected_column_2, title=f'{selected_column_2} por Tempo', labels={'Tempo': 'segundos', selected_column_2: selected_column_2}, trendline='ols')
    fig_2.update_traces(marker=dict(size=3), marker_color='darkblue')
    
    
    # Gráfico 3
    fig_3 = px.scatter(df_novo_1, x='Tempo', y=selected_column_3, title=f'{selected_column_3} por Tempo', labels={'Tempo': 'segundos', selected_column_3: selected_column_3})
    fig_3.update_traces(marker=dict(size=3), marker_color='darkblue')

    # Gráfico 4
    fig_4 = px.scatter(df_novo_2, x='Tempo', y=selected_column_4, title=f'Consumo por Tempo', labels={'Tempo': 'segundos', selected_column_4: selected_column_4})
    fig_4.update_traces(marker=dict(size=3), marker_color='darkblue')

    return fig_1, fig_2, fig_3, fig_4


# Callback para calcular e exibir média e desvio padrão do gráfico 1
@app.callback(
    Output('statistics-1', 'children'),
    [Input('column-dropdown-1', 'value'),
     Input('rotation-interval-dropdown', 'value')]
)
def update_statistics_1(selected_column, selected_categories):
    if not selected_categories or not selected_column:
        return ''

    conditions = [df['ROTACAO (RPM)'].between(*map(int, category.split('-'))) for category in selected_categories if category != 'outros']
    if 'outros' in selected_categories:
        if conditions:
            selected_data = df[~np.logical_or.reduce(conditions)][selected_column]
        else:
            selected_data = df[selected_column]
    else:
        selected_data = df[np.logical_or.reduce(conditions)][selected_column]

    mean_value = selected_data.mean()
    std_deviation = selected_data.std()

    return html.Div([
        html.Label(f'Média de {selected_column}: {mean_value:.2f}', style={'fontSize': '18px', 'fontWeight': 'bold', 'color': '#1f77b4'}),
        html.Br(),
        html.Label(f'Desvio Padrão de {selected_column}: {std_deviation:.2f}', style={'fontSize': '18px', 'fontWeight': 'bold','color': '#1f77b4'})
    ])

# Callback para calcular e exibir média e desvio padrão do gráfico 2
@app.callback(
    Output('statistics-2', 'children'),
    [Input('column-dropdown-2', 'value'),
     Input('rotation-interval-dropdown', 'value')]
)
def update_statistics_2(selected_column, selected_categories):
    if not selected_categories or not selected_column:
        return ''

    conditions = [df['ROTACAO (RPM)'].between(*map(int, category.split('-'))) for category in selected_categories if category != 'outros']
    if 'outros' in selected_categories:
        if conditions:
            selected_data = df[~np.logical_or.reduce(conditions)][selected_column]
        else:
            selected_data = df[selected_column]
    else:
        selected_data = df[np.logical_or.reduce(conditions)][selected_column]

    mean_value = selected_data.mean()
    std_deviation = selected_data.std()

    return html.Div([
        html.Label(f'Média de {selected_column}: {mean_value:.2f}', style={'fontSize': '18px', 'fontWeight': 'bold', 'color': '#1f77b4'}),
        html.Br(),
        html.Label(f'Desvio Padrão de {selected_column}: {std_deviation:.2f}', style={'fontSize': '18px', 'fontWeight': 'bold','color': '#1f77b4'})
    ])

# Callback para calcular e exibir média e desvio padrão do gráfico 3
@app.callback(
    Output('statistics-3', 'children'),
    [Input('column-dropdown-3', 'value')]
)
def update_statistics_3(selected_column):
    if not selected_column:
        return ''

    selected_data = df_novo_1[selected_column][1:]
    mean_value = selected_data.mean()
    std_deviation = selected_data.std()

    return html.Div([
        html.Label(f'Média de {selected_column}: {mean_value:.2f}', style={'fontSize': '18px', 'fontWeight': 'bold', 'color': '#1f77b4'}),
        html.Br(),
        html.Label(f'Desvio Padrão de {selected_column}: {std_deviation:.2f}', style={'fontSize': '18px', 'fontWeight': 'bold','color': '#1f77b4'})
    ])

# Rodar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)


In [170]:
import pyautogui

pyautogui.hotkey('alt','end')